# Building a Supercharged Web Agent
We’ll be building a “Supercharged Research Agent” that takes a user prompt, searches the web, and returns a clean, structured summary for human use.

In [21]:
# Import Dependency Libs
from dotenv import load_dotenv
from agents import Agent, WebSearchTool, trace, Runner
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown


load_dotenv(override=True)

# Agent Instructions: Clear and Direct
RESEARCH_PROMPT = (
    "You are a highly efficient research assistant. "
    "Given a topic, perform a thorough web search and return a well-organized, factual summary "
    "in under 500 words. Summarize clearly, capture core insights, and omit fluff. "
    "No personal opinions or commentary. This is for someone writing a report."
)

# Initialize Web Search Agent
web_search_agent = Agent(
    name="Web Research Agent",
    instructions=RESEARCH_PROMPT,
    tools=[WebSearchTool(search_context_size='low')],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
# Topic to Search
search="Best 3 performing index funds in the US market in current quarter"

# Run the Agent
with trace("Web Search Execution"):
    result = await Runner.run(web_search_agent, search)

# Display Results
display(Markdown(result.final_output))

### Pricing of OpenAI APIs
https://platform.openai.com/docs/pricing#web-search

### Check out the trace:

https://platform.openai.com/traces

## Planning Web Searches with Structured AI Output

In [ ]:
from pydantic import BaseModel

# Define structured output for each search item
class SearchTask(BaseModel):
    reason: str  # Why this particular search matters
    query: str   # The actual web search string

# The full output schema from the planner
class SearchPlan(BaseModel):
    searches: list[SearchTask]  # A list of search instructions

# How many searches should the planner generate?
NUMBER_OF_SEARCHES=10

# Instructions: Well-formed prompt for planning
PLANNER_PROMPT = (
    f"You are a financial research planner. Given a user query, identify {NUMBER_OF_SEARCHES} distinct search terms "
    "that would help answer it comprehensively. For each search, include a short reason explaining its relevance."
)

# Agent to generate the structured search plan
search_planner_agent = Agent(
    name="SearchPlannerAgent",
    instructions=PLANNER_PROMPT,
    model="gpt-4o-mini",
    output_type=SearchPlan  # Agent is expected to output a Pydantic structure
)


In [23]:
# Example financial query
user_query = "Best 3 performing index funds in the US market in current quarter"

# Execute agent and capture trace
with trace("SearchPlan Generation"):
    result = await Runner.run(search_planner_agent, user_query)
    print(result.final_output)

searches=[SearchTask(reason="To identify top-performing funds, it's essential to find resources that track and report the performance of index funds specifically for the current quarter.", query='top performing index funds US Q3 2023'), SearchTask(reason='Analyzing recent market trends provides insight into which index funds have gained traction and performed well over the specified time period.', query='US market index fund performance trends Q3 2023'), SearchTask(reason='Reviewing expert analyses and ratings on index funds helps to determine which funds are being highlighted as top performers based on various metrics.', query='expert analysis best performing index funds Q3 2023'), SearchTask(reason='Official reports or databases can provide authoritative data on fund performance metrics and rankings for the most accurate and updated information.', query='index fund performance reports Q3 2023')]


## Designing End-to-End Research Pipelines

In [36]:
from agents import function_tool
from sendgrid.helpers.mail import Email, To, Content, Mail
import sendgrid, os

# Email Sender Tool
@function_tool
def send_email(subject: str, html_body: str):
     """Send an HTML email with subject and body to the recipient list."""
     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
     sender = Email("anshulc55@gmail.com")
     recipient = To("anshulc55@icloud.com")
     content = Content("text/html", html_body)
     email = Mail(sender, recipient, subject, content).get()
     response = sg.client.mail.send.post(request_body=email)
     return {"status": "Success" , "code": response.status_code}

In [37]:
###  This agent takes a full research report, converts it into styled HTML, and triggers send_email.

# Email Agent
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="EmailAgent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [38]:
## This agent takes results from all search tasks and writes a long-form, structured markdown report.

class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""

# Report Generator Agent
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5–10 pages of content, at least 1000 words."
)

writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)

## Async Pipeline Functions

In [ ]:
## First generate a reasoned plan before searching.
import asyncio
async def search_planner(query: str):
    """Use planner_agent to generate a structured list of web searches for a given research query."""
    print("Planning for searches...")
    result = await Runner.run(search_planner_agent, query)
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

## Function to Perform WebSearch : OpenAI WebSearch Tool
async def web_search(search_task: SearchTask):
    """Run a single web search using the search_agent for a given query and reason."""
    input = f"Search term: {search_task.query}\nReason for searching: {search_task.reason}"
    result = await Runner.run(web_search_agent, input)
    return result.final_output

## Running web_search in parallel
async def perform_parallel_searches(search_plan: SearchPlan):
    """Perform web searches in parallel using asyncio to speed up data collection."""
    print("Perform Parallel Searching...")
    tasks = [asyncio.create_task(web_search(task)) for task in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching !!!")
    return results

In [40]:
## Generate Report from Search Results
async def generate_detailed_report(user_query: str, search_summaries: list[str]):
    """Use the WriterAgent to compile a structured, markdown-based research report."""
    print("Thinking deeply about the report...")
    prompt_input = f"Original query: {user_query}\nSummarized search results: {search_summaries}"
    result = await Runner.run(writer_agent, prompt_input)
    print("Finished writing the report!")
    return result.final_output

## Email the Final Report
async def dispatch_report_via_email(report: ReportData):
    """Use the EmailAgent to send the formatted report via HTML email."""
    print("Preparing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent successfully!")
    return report

In [ ]:
# Final Pipeline Execution

# Research Query
research_query = "Comparison of growth vs value investing strategies."

# Track the entire flow
with trace("Financial Research trace"):
    print("Starting full research pipeline...")

    # Step 1: Plan web searches
    search_plan = await search_planner(research_query)

    # Step 2: Perform all searches asynchronously
    search_results = await perform_parallel_searches(search_plan)

    # Step 3: Generate structured markdown report
    final_report = await generate_detailed_report(research_query, search_results)

    # Step 4: Email the report as HTML
    await dispatch_report_via_email(final_report)

    print("🎉 Research workflow completed! Check your inbox.")

Starting full research pipeline...
Planning for searches...
Will perform 20 searches
Perform Parallel Searching...
Finished searching !!!
Thinking deeply about the report...
Finished writing the report!
Preparing email...
Email sent successfully!
🎉 Research workflow completed! Check your inbox.


<table style="margin: 0; text-align: left; width:100%"> <tr> <td> <h2 style="color:#00cc00;">🌟 Congratulations!</h2> <span style="color:#00cc00;"> You’ve just designed, built, and executed a complete **Autonomous AI Research Pipeline**.<br/><br/> You didn’t just write prompts—you orchestrated agents, async tasks, and output delivery with production-level patterns.<br/><br/> 🧠 You’ve upskilled.<br/> 💼 You’ve unlocked automation possibilities.<br/> 🛠️ You’ve created reusable agent components.<br/><br/> If you're enjoying this journey, I’d love your support:<br/> 👉 Rate the course on Udemy if you haven’t already<br/> 👉 <a href="https://www.linkedin.com/in/anshulchauhan/">Connect with me on LinkedIn</a><br/></span> </td> </tr> </table>